<a href="https://colab.research.google.com/github/Shashank65536/Insurance-Prediction-Project/blob/main/LR_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Entry**

In [2]:
import os
import math
import csv
import collections
import pdb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor

%matplotlib inline

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
df = pd.read_csv('/content/drive/MyDrive/Big_Data/project_final/training_data.csv')

<ipython-input-39-338fde2d5056>:1: DtypeWarning: Columns (6,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Big_Data/project_final/training_data.csv')


In [5]:
df.head(10)

,PolicyNo,Policy_Company,Policy_Installment_Term,Policy_Billing_Code,Policy_Method_Of_Payment,Policy_Reinstatement_Fee_Indicator,Policy_Zip_Code_Garaging_Location,Vehicle_Territory,Vehicle_Make_Year,Vehicle_Make_Description,...,EEA_Prior_Bodily_Injury_Limit,EEA_PolicyYear,SYS_Renewed,SYS_New_Business,Annual_Premium,Claim_Count,Loss_Amount,Frequency,Severity,Loss_Ratio
0,164532941,Standard,6,Direct Billed to Insured,Pre-paid,N,43046,35,2004,BUIK LESABRE LI,...,100-400,2006,Y,N,320.12,0,0.0,0.0,0.0,0.0
1,164533241,Standard,6,Direct Billed to Insured,Pre-paid,N,Unknown,35,1980,CADILLAC 4-DOOR,...,100-200,2006,Y,N,259.70,0,0.0,0.0,0.0,0.0
2,164534633,Standard,6,Direct Billed to Insured,Pre-paid,N,43555,17,2005,PONT MONTANA SV,...,100-400,2006,Y,N,613.74,0,0.0,0.0,0.0,0.0
3,164534839,Standard,6,Direct Billed to Insured,Pre-paid,N,43561,17,2005,MERC GRAND MARQ,...,40-100,2006,Y,N,541.66,0,0.0,0.0,0.0,0.0
4,164534840,Standard,6,Direct Billed to Insured,Pre-paid,N,43561,17,2005,MERC GRAND MARQ,...,40-100,2006,Y,N,541.66,0,0.0,0.0,0.0,0.0
5,164534933,Standard,6,Direct Billed to Insured,Pre-paid,N,43052,27,1985,GMC S15,...,20-50,2006,Y,N,164.30,0,0.0,0.0,0.0,0.0
6,164542135,Standard,6,Direct Billed to Insured,Installment,N,44095,31,2002,FORD EXPLOR XLT,...,NaN,2006,Y,N,445.20,0,0.0,0.0,0.0,0.0
7,164542433,Standard,6,Direct Billed to Insured,Pre-paid,N,42858,35,2005,FORD F-150 SUPE,...,100-400,2006,Y,N,336.02,0,0.0,0.0,0.0,0.0
8,164543341,Standard,6,Direct Billed to Insured,Pre-paid,Y,42980,30,1991,CHEV S10 BLAZER,...,40-100,2006,Y,N,103.88,0,0.0,0.0,0.0,0.0
9,164544226,Standard,6,Direct Billed to Insured,Pre-paid,N,42597,30,2005,CADI DEVILLE,...,100-200,2006,Y,N,505.62,0,0.0,0.0,0.0,0.0


In [6]:
df.shape

(424431, 69)

In [7]:
df.columns

Index(['PolicyNo', 'Policy_Company', 'Policy_Installment_Term',
       'Policy_Billing_Code', 'Policy_Method_Of_Payment',
       'Policy_Reinstatement_Fee_Indicator',
       'Policy_Zip_Code_Garaging_Location', 'Vehicle_Territory',
       'Vehicle_Make_Year', 'Vehicle_Make_Description', 'Vehicle_Performance',
       'Vehicle_New_Cost_Amount', 'Vehicle_Symbol',
       'Vehicle_Number_Of_Drivers_Assigned', 'Vehicle_Usage',
       'Vehicle_Miles_To_Work', 'Vehicle_Days_Per_Week_Driven',
       'Vehicle_Annual_Miles', 'Vehicle_Anti_Theft_Device',
       'Vehicle_Passive_Restraint', 'Vehicle_Age_In_Years',
       'Vehicle_Med_Pay_Limit', 'Vehicle_Bodily_Injury_Limit',
       'Vehicle_Physical_Damage_Limit',
       'Vehicle_Comprehensive_Coverage_Indicator',
       'Vehicle_Comprehensive_Coverage_Limit',
       'Vehicle_Collision_Coverage_Indicator',
       'Vehicle_Collision_Coverage_Deductible', 'Driver_Total',
       'Driver_Total_Male', 'Driver_Total_Female', 'Driver_Total_Single',
     

# **Support Function**

In [8]:
def getSelectFeatures(filePath):
    with open(filePath) as f:
        featureList = f.read().splitlines()
        return featureList

In [9]:
def stripSpaces(df):
    columns = list(df.columns)
    for column in columns:
        df[column] = df[column].str.strip()
    return df

In [10]:
def filterFeatures(dataset, cols):
    catFeatures = []
    numFeatures = []
    for column in cols:
        # print(column)
        if dataset[column].dtype == 'object':
            dataset[column] = dataset[column].astype(str)
            catFeatures.append(column)
        else:
            numFeatures.append(column)
    print('columns filtered')
    return numFeatures, catFeatures

# **Feature Preperation**

In [11]:
numFeatures, catFeatures = filterFeatures(df,list(df.columns))

columns filtered


In [12]:
len(numFeatures)

42

In [13]:
df[numFeatures]

,PolicyNo,Policy_Installment_Term,Vehicle_Territory,Vehicle_Make_Year,Vehicle_New_Cost_Amount,Vehicle_Symbol,Vehicle_Number_Of_Drivers_Assigned,Vehicle_Miles_To_Work,Vehicle_Days_Per_Week_Driven,Vehicle_Age_In_Years,...,Driver_Total_Upper_Senior_Ages_70_plus,Vehicle_Driver_Points,EEA_Policy_Tenure,EEA_PolicyYear,Annual_Premium,Claim_Count,Loss_Amount,Frequency,Severity,Loss_Ratio
0,164532941,6,35,2004,-1,11,1,-1,5,3,...,1,0,22.7,2006,320.12,0,0.0,0.0,0.0,0.0
1,164533241,6,35,1980,-1,11,1,-1,5,9,...,1,0,47.1,2006,259.70,0,0.0,0.0,0.0,0.0
2,164534633,6,17,2005,-1,10,1,-1,5,2,...,1,0,47.2,2006,613.74,0,0.0,0.0,0.0,0.0
3,164534839,6,17,2005,-1,12,1,-1,5,2,...,1,0,46.7,2006,541.66,0,0.0,0.0,0.0,0.0
4,164534840,6,17,2005,-1,12,1,-1,5,2,...,1,0,47.2,2006,541.66,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424426,381713000,6,35,1999,-1,8,2,-1,5,9,...,0,0,0.0,2006,162.55,0,0.0,0.0,0.0,0.0
424427,381735600,6,31,2000,-1,8,2,-1,5,8,...,0,0,0.0,2006,117.13,0,0.0,0.0,0.0,0.0
424428,382057400,6,35,1997,-1,8,1,-1,5,9,...,0,0,0.0,2006,118.21,0,0.0,0.0,0.0,0.0
424429,382162500,6,31,1998,-1,17,1,-1,5,9,...,0,0,0.0,2006,103.93,0,0.0,0.0,0.0,0.0


In [14]:
len(catFeatures)

27

In [ ]:
df[catFeatures]

In [16]:
for col in df[catFeatures]:
    print(f"{col}:\t\t {[df[col].unique()]}\t\t cats:{len(df[col].unique())} ")

Policy_Company:		 [array(['Standard ', 'Preferred'], dtype=object)]		 cats:2 
Policy_Billing_Code:		 [array(['Direct Billed to Insured', 'Premium Finance'], dtype=object)]		 cats:2 
Policy_Method_Of_Payment:		 [array(['Pre-paid', 'Installment'], dtype=object)]		 cats:2 
Policy_Reinstatement_Fee_Indicator:		 [array(['N', 'Y'], dtype=object)]		 cats:2 
Policy_Zip_Code_Garaging_Location:		 [array(['43046', 'Unknown', '43555', ..., '14344', '43191', '82216'],
      dtype=object)]		 cats:1640 
Vehicle_Make_Description:		 [array(['BUIK LESABRE LI  ', 'CADILLAC   4-DOOR', 'PONT MONTANA SV  ', ...,
       'SUZUKI FORENZ    ', 'HOND CIVIC EX 4 D', 'CHEV LUMINA VAN  '],
      dtype=object)]		 cats:7650 
Vehicle_Performance:		 [array(['Standard', 'Intermediate', 'High', 'Sports', 'Sports Premium'],
      dtype=object)]		 cats:5 
Vehicle_Usage:		 [array(['Farm    ', 'Pleasure', 'Work    ', 'Business'], dtype=object)]		 cats:4 
Vehicle_Annual_Miles:		 [array(['Unknown', '0', '30000', '50000', '2000

In [17]:
unknownColumns = []
dualCatFeature = []
someCatFeature = []
highCatFeature = []

for col in df[catFeatures]:
    if df[col].astype(str).str.contains('Unknown').any():
        unknownColumns.append(col)
    else:
        if len(df[col].unique()) <= 2:
            dualCatFeature.append(col)
        elif (len(df[col].unique()) > 2) and (len(df[col].unique()) <= 5) :
            someCatFeature.append(col)
        else:
            highCatFeature.append(col)

print("Columns with Unknown Values: ",unknownColumns)
print("Columns with 2 Values: ",dualCatFeature)
print("Columns with some Values: ",someCatFeature)
print("Columns with several categories: ",highCatFeature)

Columns with Unknown Values:  ['Policy_Zip_Code_Garaging_Location', 'Vehicle_Annual_Miles', 'Vehicle_Anti_Theft_Device', 'Vehicle_Passive_Restraint', 'EEA_Policy_Zip_Code_3']
Columns with 2 Values:  ['Policy_Company', 'Policy_Billing_Code', 'Policy_Method_Of_Payment', 'Policy_Reinstatement_Fee_Indicator', 'Vehicle_Comprehensive_Coverage_Indicator', 'Vehicle_Collision_Coverage_Indicator', 'Vehicle_Youthful_Driver_Indicator', 'Vehicle_Youthful_Good_Student_Code', 'Vehicle_Safe_Driver_Discount_Indicator', 'EEA_Liability_Coverage_Only_Indicator', 'EEA_Multi_Auto_Policies_Indicator', 'EEA_Packaged_Policy_Indicator', 'EEA_Full_Coverage_Indicator', 'SYS_Renewed', 'SYS_New_Business']
Columns with some Values:  ['Vehicle_Performance', 'Vehicle_Usage', 'Vehicle_Youthful_Driver_Training_Code', 'EEA_Agency_Type']
Columns with several categories:  ['Vehicle_Make_Description', 'Vehicle_Bodily_Injury_Limit', 'EEA_Prior_Bodily_Injury_Limit']


In [18]:
catFeatures = dualCatFeature
catFeatures

['Policy_Company',
 'Policy_Billing_Code',
 'Policy_Method_Of_Payment',
 'Policy_Reinstatement_Fee_Indicator',
 'Vehicle_Comprehensive_Coverage_Indicator',
 'Vehicle_Collision_Coverage_Indicator',
 'Vehicle_Youthful_Driver_Indicator',
 'Vehicle_Youthful_Good_Student_Code',
 'Vehicle_Safe_Driver_Discount_Indicator',
 'EEA_Liability_Coverage_Only_Indicator',
 'EEA_Multi_Auto_Policies_Indicator',
 'EEA_Packaged_Policy_Indicator',
 'EEA_Full_Coverage_Indicator',
 'SYS_Renewed',
 'SYS_New_Business']

In [19]:
df_cat = df[catFeatures]
# df_cat = stripSpaces(df_cat)
print('shape',df_cat.shape,'\n')
df_cat.head(10)

shape (424431, 15) 



,Policy_Company,Policy_Billing_Code,Policy_Method_Of_Payment,Policy_Reinstatement_Fee_Indicator,Vehicle_Comprehensive_Coverage_Indicator,Vehicle_Collision_Coverage_Indicator,Vehicle_Youthful_Driver_Indicator,Vehicle_Youthful_Good_Student_Code,Vehicle_Safe_Driver_Discount_Indicator,EEA_Liability_Coverage_Only_Indicator,EEA_Multi_Auto_Policies_Indicator,EEA_Packaged_Policy_Indicator,EEA_Full_Coverage_Indicator,SYS_Renewed,SYS_New_Business
0,Standard,Direct Billed to Insured,Pre-paid,N,N,Y,N,Not Eligible for Good Student Credit,Y,N,N,N,Y,Y,N
1,Standard,Direct Billed to Insured,Pre-paid,N,N,Y,N,Not Eligible for Good Student Credit,Y,N,N,N,Y,Y,N
2,Standard,Direct Billed to Insured,Pre-paid,N,N,Y,N,Not Eligible for Good Student Credit,Y,N,N,N,Y,Y,N
3,Standard,Direct Billed to Insured,Pre-paid,N,N,Y,N,Not Eligible for Good Student Credit,Y,N,Y,Y,Y,Y,N
4,Standard,Direct Billed to Insured,Pre-paid,N,N,Y,N,Not Eligible for Good Student Credit,Y,N,Y,Y,Y,Y,N
5,Standard,Direct Billed to Insured,Pre-paid,N,N,N,N,Not Eligible for Good Student Credit,Y,N,Y,Y,N,Y,N
6,Standard,Direct Billed to Insured,Installment,N,Y,Y,N,Not Eligible for Good Student Credit,N,N,Y,Y,Y,Y,N
7,Standard,Direct Billed to Insured,Pre-paid,N,N,Y,N,Not Eligible for Good Student Credit,Y,N,Y,Y,Y,Y,N
8,Standard,Direct Billed to Insured,Pre-paid,Y,N,N,N,Not Eligible for Good Student Credit,Y,Y,Y,N,N,Y,N
9,Standard,Direct Billed to Insured,Pre-paid,N,N,Y,N,Not Eligible for Good Student Credit,Y,N,Y,Y,Y,Y,N


In [20]:
#Categorical Encoding
df_cat_oh = pd.get_dummies(df_cat)

In [21]:
print(df_cat_oh.shape)
oh_cols = list(df_cat_oh.columns)
df_cat_oh.columns

(424431, 30)


Index(['Policy_Company_Preferred', 'Policy_Company_Standard ',
       'Policy_Billing_Code_Direct Billed to Insured',
       'Policy_Billing_Code_Premium Finance',
       'Policy_Method_Of_Payment_Installment',
       'Policy_Method_Of_Payment_Pre-paid',
       'Policy_Reinstatement_Fee_Indicator_N',
       'Policy_Reinstatement_Fee_Indicator_Y',
       'Vehicle_Comprehensive_Coverage_Indicator_N',
       'Vehicle_Comprehensive_Coverage_Indicator_Y',
       'Vehicle_Collision_Coverage_Indicator_N',
       'Vehicle_Collision_Coverage_Indicator_Y',
       'Vehicle_Youthful_Driver_Indicator_N',
       'Vehicle_Youthful_Driver_Indicator_Y',
       'Vehicle_Youthful_Good_Student_Code_Eligible for Good Student Credit    ',
       'Vehicle_Youthful_Good_Student_Code_Not Eligible for Good Student Credit',
       'Vehicle_Safe_Driver_Discount_Indicator_N',
       'Vehicle_Safe_Driver_Discount_Indicator_Y',
       'EEA_Liability_Coverage_Only_Indicator_N',
       'EEA_Liability_Coverage_Only_Ind

In [ ]:
# target_column = 'Loss_Amount'

# correlations = {}

# for col in targetless_cols:
#     # Calculate point-biserial correlation coefficient
#     correlation, _ = stats.pointbiserialr(df_cat_oh[col], df_cat_oh[target_column])
#     # Store the correlation in the dictionary
#     correlations[col] = correlation

# # Convert the dictionary to a DataFrame for easier analysis
# correlation_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Correlation'])

# # Sort the DataFrame by correlation values
# correlation_df = correlation_df.sort_values(by='Correlation', ascending=False)

# # Display the correlation DataFrame
# print(correlation_df)


In [22]:
targetColumn = ['Loss_Amount']
numFeatures = [feature for feature in numFeatures if feature not in ['Loss_Amount','PolicyNo','Loss_Ratio']]
numFeatures1 = numFeatures + targetColumn
df_num = df[numFeatures]
df_num1 = df[numFeatures1]

In [23]:
print(df_num.columns)
print(df_num1.columns)

Index(['Policy_Installment_Term', 'Vehicle_Territory', 'Vehicle_Make_Year',
       'Vehicle_New_Cost_Amount', 'Vehicle_Symbol',
       'Vehicle_Number_Of_Drivers_Assigned', 'Vehicle_Miles_To_Work',
       'Vehicle_Days_Per_Week_Driven', 'Vehicle_Age_In_Years',
       'Vehicle_Med_Pay_Limit', 'Vehicle_Physical_Damage_Limit',
       'Vehicle_Comprehensive_Coverage_Limit',
       'Vehicle_Collision_Coverage_Deductible', 'Driver_Total',
       'Driver_Total_Male', 'Driver_Total_Female', 'Driver_Total_Single',
       'Driver_Total_Married', 'Driver_Total_Related_To_Insured_Self',
       'Driver_Total_Related_To_Insured_Spouse',
       'Driver_Total_Related_To_Insured_Child',
       'Driver_Total_Licensed_In_State', 'Driver_Minimum_Age',
       'Driver_Maximum_Age', 'Driver_Total_Teenager_Age_15_19',
       'Driver_Total_College_Ages_20_23',
       'Driver_Total_Young_Adult_Ages_24_29',
       'Driver_Total_Low_Middle_Adult_Ages_30_39',
       'Driver_Total_Middle_Adult_Ages_40_49', 'Driver_

In [24]:
missingValues = df_num.isna().sum()
missingValues

Policy_Installment_Term                     0
Vehicle_Territory                           0
Vehicle_Make_Year                           0
Vehicle_New_Cost_Amount                     0
Vehicle_Symbol                              0
Vehicle_Number_Of_Drivers_Assigned          0
Vehicle_Miles_To_Work                       0
Vehicle_Days_Per_Week_Driven                0
Vehicle_Age_In_Years                        0
Vehicle_Med_Pay_Limit                       0
Vehicle_Physical_Damage_Limit               0
Vehicle_Comprehensive_Coverage_Limit        0
Vehicle_Collision_Coverage_Deductible       0
Driver_Total                                0
Driver_Total_Male                           0
Driver_Total_Female                         0
Driver_Total_Single                         0
Driver_Total_Married                        0
Driver_Total_Related_To_Insured_Self        0
Driver_Total_Related_To_Insured_Spouse      0
Driver_Total_Related_To_Insured_Child       0
Driver_Total_Licensed_In_State    

In [25]:
df_num1

,Policy_Installment_Term,Vehicle_Territory,Vehicle_Make_Year,Vehicle_New_Cost_Amount,Vehicle_Symbol,Vehicle_Number_Of_Drivers_Assigned,Vehicle_Miles_To_Work,Vehicle_Days_Per_Week_Driven,Vehicle_Age_In_Years,Vehicle_Med_Pay_Limit,...,Driver_Total_Senior_Ages_65_69,Driver_Total_Upper_Senior_Ages_70_plus,Vehicle_Driver_Points,EEA_Policy_Tenure,EEA_PolicyYear,Annual_Premium,Claim_Count,Frequency,Severity,Loss_Amount
0,6,35,2004,-1,11,1,-1,5,3,1000,...,0,1,0,22.7,2006,320.12,0,0.0,0.0,0.0
1,6,35,1980,-1,11,1,-1,5,9,2000,...,0,1,0,47.1,2006,259.70,0,0.0,0.0,0.0
2,6,17,2005,-1,10,1,-1,5,2,50000,...,0,1,0,47.2,2006,613.74,0,0.0,0.0,0.0
3,6,17,2005,-1,12,1,-1,5,2,5000,...,0,1,0,46.7,2006,541.66,0,0.0,0.0,0.0
4,6,17,2005,-1,12,1,-1,5,2,5000,...,0,1,0,47.2,2006,541.66,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424426,6,35,1999,-1,8,2,-1,5,9,5000,...,0,0,0,0.0,2006,162.55,0,0.0,0.0,0.0
424427,6,31,2000,-1,8,2,-1,5,8,5000,...,0,0,0,0.0,2006,117.13,0,0.0,0.0,0.0
424428,6,35,1997,-1,8,1,-1,5,9,50000,...,0,0,0,0.0,2006,118.21,0,0.0,0.0,0.0
424429,6,31,1998,-1,17,1,-1,5,9,5000,...,0,0,0,0.0,2006,103.93,0,0.0,0.0,0.0


In [26]:
# Calculate correlations
correlation_matrix = df_num1.corr()

# Extract correlations with the target column
correlation_with_target = correlation_matrix['Loss_Amount']

# Sort correlations in descending order
sorted_correlation_with_target = correlation_with_target.abs().sort_values(ascending = False)

# Display correlations
print(sorted_correlation_with_target)

Loss_Amount                                 1.000000
Severity                                    0.993720
Claim_Count                                 0.319414
Frequency                                   0.146085
Annual_Premium                              0.032097
Vehicle_Age_In_Years                        0.028379
Vehicle_Make_Year                           0.027465
Vehicle_Symbol                              0.021756
Vehicle_Collision_Coverage_Deductible       0.020509
Driver_Total_Teenager_Age_15_19             0.019951
Driver_Minimum_Age                          0.017122
Policy_Installment_Term                     0.015728
Driver_Maximum_Age                          0.015290
Driver_Total_Related_To_Insured_Child       0.014927
Driver_Total_Single                         0.014020
Vehicle_Driver_Points                       0.012576
EEA_Policy_Tenure                           0.012197
Driver_Total_Adult_Ages_50_64               0.011358
Driver_Total_College_Ages_20_23             0.

In [27]:
selected_num_features = ['Severity', 'Claim_Count','Frequency','Annual_Premium','Vehicle_Age_In_Years',
                         'Vehicle_Make_Year','Vehicle_Symbol','Vehicle_Collision_Coverage_Deductible'
                         ,'Driver_Minimum_Age','Driver_Maximum_Age','Policy_Installment_Term'
                         ,'Vehicle_Driver_Points','EEA_Policy_Tenure','Driver_Total_Married','Driver_Total_Single'
                         ,'Driver_Total_Female','Driver_Total_Male','Vehicle_Med_Pay_Limit','Vehicle_Miles_To_Work',
                         'Driver_Total_Teenager_Age_15_19','Driver_Total_College_Ages_20_23','Driver_Total_Young_Adult_Ages_24_29',
                         'Driver_Total_Low_Middle_Adult_Ages_30_39','Driver_Total_Middle_Adult_Ages_40_49','Driver_Total_Adult_Ages_50_64',
                         'Driver_Total_Senior_Ages_65_69','Driver_Total_Upper_Senior_Ages_70_plus','Loss_Amount']

df_selected = df_num1[selected_num_features]
df_selected

,Severity,Claim_Count,Frequency,Annual_Premium,Vehicle_Age_In_Years,Vehicle_Make_Year,Vehicle_Symbol,Vehicle_Collision_Coverage_Deductible,Driver_Minimum_Age,Driver_Maximum_Age,...,Vehicle_Miles_To_Work,Driver_Total_Teenager_Age_15_19,Driver_Total_College_Ages_20_23,Driver_Total_Young_Adult_Ages_24_29,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Middle_Adult_Ages_40_49,Driver_Total_Adult_Ages_50_64,Driver_Total_Senior_Ages_65_69,Driver_Total_Upper_Senior_Ages_70_plus,Loss_Amount
0,0.0,0,0.0,320.12,3,2004,11,500,75,75,...,-1,0,0,0,0,0,0,0,1,0.0
1,0.0,0,0.0,259.70,9,1980,11,50,71,71,...,-1,0,0,0,0,0,0,0,1,0.0
2,0.0,0,0.0,613.74,2,2005,10,100,80,80,...,-1,0,0,0,0,0,0,0,1,0.0
3,0.0,0,0.0,541.66,2,2005,12,250,74,74,...,-1,0,0,0,0,0,0,0,1,0.0
4,0.0,0,0.0,541.66,2,2005,12,250,75,75,...,-1,0,0,0,0,0,0,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424426,0.0,0,0.0,162.55,9,1999,8,-1,18,18,...,-1,1,0,0,0,0,0,0,0,0.0
424427,0.0,0,0.0,117.13,8,2000,8,-1,20,20,...,-1,0,1,0,0,0,0,0,0,0.0
424428,0.0,0,0.0,118.21,9,1997,8,500,58,58,...,-1,0,0,0,0,0,1,0,0,0.0
424429,0.0,0,0.0,103.93,9,1998,17,-1,30,30,...,-1,0,0,0,1,0,0,0,0,0.0


In [28]:
df_clean = df_selected.join(df_cat_oh, how = 'outer')
print('shape: ', df_clean.shape,'\n')
print('columns: ', df_clean.columns,'\n')
df_clean.head(10)

shape:  (424431, 58) 

columns:  Index(['Severity', 'Claim_Count', 'Frequency', 'Annual_Premium',
       'Vehicle_Age_In_Years', 'Vehicle_Make_Year', 'Vehicle_Symbol',
       'Vehicle_Collision_Coverage_Deductible', 'Driver_Minimum_Age',
       'Driver_Maximum_Age', 'Policy_Installment_Term',
       'Vehicle_Driver_Points', 'EEA_Policy_Tenure', 'Driver_Total_Married',
       'Driver_Total_Single', 'Driver_Total_Female', 'Driver_Total_Male',
       'Vehicle_Med_Pay_Limit', 'Vehicle_Miles_To_Work',
       'Driver_Total_Teenager_Age_15_19', 'Driver_Total_College_Ages_20_23',
       'Driver_Total_Young_Adult_Ages_24_29',
       'Driver_Total_Low_Middle_Adult_Ages_30_39',
       'Driver_Total_Middle_Adult_Ages_40_49', 'Driver_Total_Adult_Ages_50_64',
       'Driver_Total_Senior_Ages_65_69',
       'Driver_Total_Upper_Senior_Ages_70_plus', 'Loss_Amount',
       'Policy_Company_Preferred', 'Policy_Company_Standard ',
       'Policy_Billing_Code_Direct Billed to Insured',
       'Policy_Billin

,Severity,Claim_Count,Frequency,Annual_Premium,Vehicle_Age_In_Years,Vehicle_Make_Year,Vehicle_Symbol,Vehicle_Collision_Coverage_Deductible,Driver_Minimum_Age,Driver_Maximum_Age,...,EEA_Multi_Auto_Policies_Indicator_N,EEA_Multi_Auto_Policies_Indicator_Y,EEA_Packaged_Policy_Indicator_N,EEA_Packaged_Policy_Indicator_Y,EEA_Full_Coverage_Indicator_N,EEA_Full_Coverage_Indicator_Y,SYS_Renewed_N,SYS_Renewed_Y,SYS_New_Business_N,SYS_New_Business_Y
0,0.0,0,0.0,320.12,3,2004,11,500,75,75,...,True,False,True,False,False,True,False,True,True,False
1,0.0,0,0.0,259.70,9,1980,11,50,71,71,...,True,False,True,False,False,True,False,True,True,False
2,0.0,0,0.0,613.74,2,2005,10,100,80,80,...,True,False,True,False,False,True,False,True,True,False
3,0.0,0,0.0,541.66,2,2005,12,250,74,74,...,False,True,False,True,False,True,False,True,True,False
4,0.0,0,0.0,541.66,2,2005,12,250,75,75,...,False,True,False,True,False,True,False,True,True,False
5,0.0,0,0.0,164.30,9,1985,7,-1,47,47,...,False,True,False,True,True,False,False,True,True,False
6,0.0,0,0.0,445.20,5,2002,11,500,54,60,...,False,True,False,True,False,True,False,True,True,False
7,0.0,0,0.0,336.02,2,2005,16,1000,74,85,...,False,True,False,True,False,True,False,True,True,False
8,0.0,0,0.0,103.88,9,1991,12,-1,74,74,...,False,True,True,False,True,False,False,True,True,False
9,0.0,0,0.0,505.62,2,2005,15,500,71,71,...,False,True,False,True,False,True,False,True,True,False


# **Portfolio Building**

In [29]:
selectedFeatures = list(df_clean.columns)
# selectedFeatures = [item for item in selectedFeatures if item != 'PolicyNo']
print(selectedFeatures)
print('shape:',len(selectedFeatures))

['Severity', 'Claim_Count', 'Frequency', 'Annual_Premium', 'Vehicle_Age_In_Years', 'Vehicle_Make_Year', 'Vehicle_Symbol', 'Vehicle_Collision_Coverage_Deductible', 'Driver_Minimum_Age', 'Driver_Maximum_Age', 'Policy_Installment_Term', 'Vehicle_Driver_Points', 'EEA_Policy_Tenure', 'Driver_Total_Married', 'Driver_Total_Single', 'Driver_Total_Female', 'Driver_Total_Male', 'Vehicle_Med_Pay_Limit', 'Vehicle_Miles_To_Work', 'Driver_Total_Teenager_Age_15_19', 'Driver_Total_College_Ages_20_23', 'Driver_Total_Young_Adult_Ages_24_29', 'Driver_Total_Low_Middle_Adult_Ages_30_39', 'Driver_Total_Middle_Adult_Ages_40_49', 'Driver_Total_Adult_Ages_50_64', 'Driver_Total_Senior_Ages_65_69', 'Driver_Total_Upper_Senior_Ages_70_plus', 'Loss_Amount', 'Policy_Company_Preferred', 'Policy_Company_Standard ', 'Policy_Billing_Code_Direct Billed to Insured', 'Policy_Billing_Code_Premium Finance', 'Policy_Method_Of_Payment_Installment', 'Policy_Method_Of_Payment_Pre-paid', 'Policy_Reinstatement_Fee_Indicator_N', 'P

In [30]:
df_clean_pol = df_clean[selectedFeatures]
df_clean_pol.shape

(424431, 58)

In [31]:
polPerPort = [1000, 2000, 3000, 4000, 5000]
perLosses = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20]

In [32]:
df_test_portfolios = []
for i in range(10):
    df = df_clean_pol.sample(frac=1)
    df_loss_zero = df[df['Loss_Amount'] == 0]
    df_loss_non_zero = df[df['Loss_Amount'] != 0]
    df_without_loss = 0
    df_with_loss = 0
    for i in range(len(polPerPort)):
        for j in range(len(perLosses)):
            without_loss_size = int(((100 - perLosses[j]) / 100) * polPerPort[i])
            with_loss_size = polPerPort[i] - without_loss_size

            # Calculate index ranges for slicing
            df_without_loss_end = df_without_loss + without_loss_size
            df_with_loss_end = df_with_loss + with_loss_size

            # Slice the DataFrame using iloc
            df_without_loss_slice = df_loss_zero.iloc[df_without_loss:df_without_loss_end]
            df_with_loss_slice = df_loss_non_zero.iloc[df_with_loss:df_with_loss_end]

            # Increment the index counters
            df_without_loss = df_without_loss_end
            df_with_loss = df_with_loss_end

            # Concatenate DataFrames
            df_merge = pd.concat([df_without_loss_slice, df_with_loss_slice])
            df_test_portfolios.append(df_merge)


In [33]:
df_clean_columns = ['Severity', 'Claim_Count', 'Frequency', 'Annual_Premium', 'Vehicle_Age_In_Years',
                    'Vehicle_Make_Year', 'Vehicle_Symbol', 'Vehicle_Collision_Coverage_Deductible',
                    'Driver_Minimum_Age', 'Driver_Maximum_Age', 'Policy_Installment_Term',
                    'Vehicle_Driver_Points', 'EEA_Policy_Tenure', 'Driver_Total_Married',
                    'Driver_Total_Single', 'Driver_Total_Female', 'Driver_Total_Male',
                    'Vehicle_Med_Pay_Limit', 'Vehicle_Miles_To_Work', 'Driver_Total_Teenager_Age_15_19',
                    'Driver_Total_College_Ages_20_23', 'Driver_Total_Young_Adult_Ages_24_29',
                    'Driver_Total_Low_Middle_Adult_Ages_30_39', 'Driver_Total_Middle_Adult_Ages_40_49',
                    'Driver_Total_Adult_Ages_50_64', 'Driver_Total_Senior_Ages_65_69',
                    'Driver_Total_Upper_Senior_Ages_70_plus', 'Loss_Amount', 'Policy_Company_Preferred',
                    'Policy_Company_Standard ', 'Policy_Billing_Code_Direct Billed to Insured',
                    'Policy_Billing_Code_Premium Finance', 'Policy_Method_Of_Payment_Installment',
                    'Policy_Method_Of_Payment_Pre-paid', 'Policy_Reinstatement_Fee_Indicator_N',
                    'Policy_Reinstatement_Fee_Indicator_Y', 'Vehicle_Comprehensive_Coverage_Indicator_N',
                    'Vehicle_Comprehensive_Coverage_Indicator_Y', 'Vehicle_Collision_Coverage_Indicator_N',
                    'Vehicle_Collision_Coverage_Indicator_Y', 'Vehicle_Youthful_Driver_Indicator_N',
                    'Vehicle_Youthful_Driver_Indicator_Y', 'Vehicle_Youthful_Good_Student_Code_Eligible for Good Student Credit',
                    'Vehicle_Youthful_Good_Student_Code_Not Eligible for Good Student Credit',
                    'Vehicle_Safe_Driver_Discount_Indicator_N', 'Vehicle_Safe_Driver_Discount_Indicator_Y',
                    'EEA_Liability_Coverage_Only_Indicator_N', 'EEA_Liability_Coverage_Only_Indicator_Y',
                    'EEA_Multi_Auto_Policies_Indicator_N', 'EEA_Multi_Auto_Policies_Indicator_Y', 'EEA_Packaged_Policy_Indicator_N',
                    'EEA_Packaged_Policy_Indicator_Y', 'EEA_Full_Coverage_Indicator_N', 'EEA_Full_Coverage_Indicator_Y',
                    'SYS_Renewed_N', 'SYS_Renewed_Y', 'SYS_New_Business_N', 'SYS_New_Business_Y']

df_data_final = None

sum_columns = list(df_cat_oh.columns) + ['Annual_Premium']
print(sum_columns)

mean_columns = [feature for feature in selected_num_features if feature not in ['Annual_Premium', 'Loss_Amount']]
print(mean_columns)

target_column = ['Loss_Amount']
df_target = df[target_column]

['Policy_Company_Preferred', 'Policy_Company_Standard ', 'Policy_Billing_Code_Direct Billed to Insured', 'Policy_Billing_Code_Premium Finance', 'Policy_Method_Of_Payment_Installment', 'Policy_Method_Of_Payment_Pre-paid', 'Policy_Reinstatement_Fee_Indicator_N', 'Policy_Reinstatement_Fee_Indicator_Y', 'Vehicle_Comprehensive_Coverage_Indicator_N', 'Vehicle_Comprehensive_Coverage_Indicator_Y', 'Vehicle_Collision_Coverage_Indicator_N', 'Vehicle_Collision_Coverage_Indicator_Y', 'Vehicle_Youthful_Driver_Indicator_N', 'Vehicle_Youthful_Driver_Indicator_Y', 'Vehicle_Youthful_Good_Student_Code_Eligible for Good Student Credit    ', 'Vehicle_Youthful_Good_Student_Code_Not Eligible for Good Student Credit', 'Vehicle_Safe_Driver_Discount_Indicator_N', 'Vehicle_Safe_Driver_Discount_Indicator_Y', 'EEA_Liability_Coverage_Only_Indicator_N', 'EEA_Liability_Coverage_Only_Indicator_Y', 'EEA_Multi_Auto_Policies_Indicator_N', 'EEA_Multi_Auto_Policies_Indicator_Y', 'EEA_Packaged_Policy_Indicator_N', 'EEA_Pac

In [34]:
def portfolio_to_features(portfolio, train=True):
    portfolio_mean = portfolio[mean_columns].mean()
    list_mean = portfolio_mean.to_list()

    portfolio_sum = portfolio[sum_columns].sum()
    list_sum = portfolio_sum.to_list()

    if train == True:
        list_target_sum = portfolio[target_column].sum().to_list()

    list_merge = list_mean + list_sum

    if train == True:
        list_merge = list_merge + list_target_sum

    return list_merge

In [35]:
def features_to_training_df(portfolios_list):
    features_list = mean_columns + sum_columns + target_column
    final_set = []
    for portfolio in portfolios_list:
        list_portfolio = portfolio_to_features(portfolio)
        final_set.append(list_portfolio)
    df_final_training = pd.DataFrame(final_set, columns = features_list)
    return df_final_training

In [36]:
finalTrainSet = features_to_training_df(df_test_portfolios)

In [41]:
finalTrainSet.shape
finalTrainSet.to_csv('/content/drive/MyDrive/Big_Data/project_final/final_train_set.csv', index=False)

In [43]:
finalTrainSet.head()

,Severity,Claim_Count,Frequency,Vehicle_Age_In_Years,Vehicle_Make_Year,Vehicle_Symbol,Vehicle_Collision_Coverage_Deductible,Driver_Minimum_Age,Driver_Maximum_Age,Policy_Installment_Term,...,EEA_Packaged_Policy_Indicator_N,EEA_Packaged_Policy_Indicator_Y,EEA_Full_Coverage_Indicator_N,EEA_Full_Coverage_Indicator_Y,SYS_Renewed_N,SYS_Renewed_Y,SYS_New_Business_N,SYS_New_Business_Y,Annual_Premium,Loss_Amount
0,88.354450,0.010,0.020000,7.205,1996.316,11.005,278.186,46.958,48.078,6.216,...,545.0,455.0,408.0,592.0,110.0,890.0,900.0,100.0,320430.75,88354.45
1,61.445210,0.023,0.053527,7.063,1996.807,11.138,298.909,46.082,47.098,6.192,...,542.0,458.0,387.0,613.0,122.0,878.0,901.0,99.0,331842.93,65702.05
2,67.764795,0.033,0.066215,7.086,1996.684,11.041,283.496,46.369,47.644,6.168,...,521.0,479.0,400.0,600.0,103.0,897.0,910.0,90.0,325237.87,72177.46
3,180.195765,0.042,0.089705,6.867,1997.181,11.072,302.127,46.184,47.241,6.246,...,530.0,470.0,370.0,630.0,123.0,877.0,918.0,82.0,336828.75,183858.23
4,308.031820,0.052,0.188548,7.118,1996.911,11.252,294.003,46.194,47.621,6.240,...,537.0,463.0,392.0,608.0,136.0,864.0,888.0,112.0,333532.78,310501.29


# **Predictions**

In [44]:
X_train = finalTrainSet.loc[:, finalTrainSet.columns != 'Loss_Amount']
y_train = finalTrainSet['Loss_Amount']

In [45]:
X_train.shape, y_train.shape

((550, 57), (550,))

In [46]:
X_train.columns

Index(['Severity', 'Claim_Count', 'Frequency', 'Vehicle_Age_In_Years',
       'Vehicle_Make_Year', 'Vehicle_Symbol',
       'Vehicle_Collision_Coverage_Deductible', 'Driver_Minimum_Age',
       'Driver_Maximum_Age', 'Policy_Installment_Term',
       'Vehicle_Driver_Points', 'EEA_Policy_Tenure', 'Driver_Total_Married',
       'Driver_Total_Single', 'Driver_Total_Female', 'Driver_Total_Male',
       'Vehicle_Med_Pay_Limit', 'Vehicle_Miles_To_Work',
       'Driver_Total_Teenager_Age_15_19', 'Driver_Total_College_Ages_20_23',
       'Driver_Total_Young_Adult_Ages_24_29',
       'Driver_Total_Low_Middle_Adult_Ages_30_39',
       'Driver_Total_Middle_Adult_Ages_40_49', 'Driver_Total_Adult_Ages_50_64',
       'Driver_Total_Senior_Ages_65_69',
       'Driver_Total_Upper_Senior_Ages_70_plus', 'Policy_Company_Preferred',
       'Policy_Company_Standard ',
       'Policy_Billing_Code_Direct Billed to Insured',
       'Policy_Billing_Code_Premium Finance',
       'Policy_Method_Of_Payment_Installm

In [47]:
decision_tree_regressor = DecisionTreeRegressor()
decision_tree_regressor.fit(X_train, y_train)

DecisionTreeRegressor()

In [48]:
y_train_pred = decision_tree_regressor.predict(X_train)

In [49]:
mae = mean_absolute_error(y_train, y_train_pred)
mae

0.0

In [51]:
!pip install xgboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 3.0 MB/s eta 0:00:00


In [52]:
from xgboost import XGBRegressor
xg_regressor = XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.01, max_depth = 3, alpha = 10, n_estimators = 100)
xg_regressor.fit(np.nan_to_num(X_train.to_numpy()), y_train)

XGBRegressor(alpha=10, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [53]:
y_train_pred_xg = xg_regressor.predict(np.nan_to_num(X_train.to_numpy()))

In [ ]:
y_train_pred_xg

In [55]:
mae_xg = mean_absolute_error(y_train, y_train_pred_xg)
mae_xg

365226.14181363635

In [76]:
def load_test_portfolios():
    path_testing_dataset = "/content/drive/MyDrive/Big_Data/project_final/testing_portfolios"
    portfolio_files = os.listdir(path_testing_dataset)
    portfolio_dict = {}
    for i in range(len(portfolio_files)):
        portfolio_id = int(portfolio_files[i].split('.')[0][15:])
        portfolio_dict[portfolio_id] = portfolio_files[i]
    od = collections.OrderedDict(sorted(portfolio_dict.items()))
    po_list = []
    for key,val in od.items():
        po_list.append(val)
    df_portfolio_list = []
    for portfolio_file in po_list:
        file_path = os.path.join(path_testing_dataset, portfolio_file)
        df_test_portfolio = pd.read_csv(file_path)
        test_num_features = numFeatures.copy()

        # Assuming numFeatures is a list defined somewhere in your code
        test_num_features = [feature for feature in test_num_features if feature not in ['Claim_Count', 'Frequency', 'Severity']]

        df_test_num = df_test_portfolio[test_num_features]
        df_test_cat = df_test_portfolio[catFeatures]
        df_test_cat_strip = stripSpaces(df_test_cat)
        df_test_portfolio = df_test_num.join(df_test_cat_strip, how='outer')
        df_test_portfolio = pd.get_dummies(df_test_portfolio)
        df_test_portfolio = df_test_portfolio.reindex(sorted(X_train.columns), axis=1)

        df_portfolio_list.append(df_test_portfolio)
        print(df_test_portfolio.columns)

    result_dict = collections.OrderedDict()
    for i in range(len(po_list)):
        portfolio_id = po_list[i].split('.')[0][5:]
        result_dict[portfolio_id] = df_portfolio_list[i]
    return result_dict

In [69]:
def features_to_testing_df(df_portfolios_ordered_dict):
    features_list = mean_columns + sum_columns
    features_list.insert(0, 'ID')
    final_list = []
    for portfolio_id, single_portfolio in df_portfolios_ordered_dict.items():
        ls_portfolio = portfolio_to_features(single_portfolio, False)
        ls_portfolio.insert(0, portfolio_id)
        final_list.append(ls_portfolio)
    df_final_testing = pd.DataFrame(final_list, columns = features_list)
    return df_final_testing

In [77]:
test_portfolios_dict = load_test_portfolios()

<ipython-input-9-0e40a977e25f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.strip()
<ipython-input-9-0e40a977e25f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.strip()
<ipython-input-9-0e40a977e25f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

Index(['Annual_Premium', 'Claim_Count', 'Driver_Maximum_Age',
       'Driver_Minimum_Age', 'Driver_Total_Adult_Ages_50_64',
       'Driver_Total_College_Ages_20_23', 'Driver_Total_Female',
       'Driver_Total_Low_Middle_Adult_Ages_30_39', 'Driver_Total_Male',
       'Driver_Total_Married', 'Driver_Total_Middle_Adult_Ages_40_49',
       'Driver_Total_Senior_Ages_65_69', 'Driver_Total_Single',
       'Driver_Total_Teenager_Age_15_19',
       'Driver_Total_Upper_Senior_Ages_70_plus',
       'Driver_Total_Young_Adult_Ages_24_29', 'EEA_Full_Coverage_Indicator_N',
       'EEA_Full_Coverage_Indicator_Y',
       'EEA_Liability_Coverage_Only_Indicator_N',
       'EEA_Liability_Coverage_Only_Indicator_Y',
       'EEA_Multi_Auto_Policies_Indicator_N',
       'EEA_Multi_Auto_Policies_Indicator_Y',
       'EEA_Packaged_Policy_Indicator_N', 'EEA_Packaged_Policy_Indicator_Y',
       'EEA_Policy_Tenure', 'Frequency',
       'Policy_Billing_Code_Direct Billed to Insured',
       'Policy_Billing_Code_P

<ipython-input-9-0e40a977e25f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.strip()
<ipython-input-9-0e40a977e25f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.strip()
<ipython-input-9-0e40a977e25f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

Index(['Annual_Premium', 'Claim_Count', 'Driver_Maximum_Age',
       'Driver_Minimum_Age', 'Driver_Total_Adult_Ages_50_64',
       'Driver_Total_College_Ages_20_23', 'Driver_Total_Female',
       'Driver_Total_Low_Middle_Adult_Ages_30_39', 'Driver_Total_Male',
       'Driver_Total_Married', 'Driver_Total_Middle_Adult_Ages_40_49',
       'Driver_Total_Senior_Ages_65_69', 'Driver_Total_Single',
       'Driver_Total_Teenager_Age_15_19',
       'Driver_Total_Upper_Senior_Ages_70_plus',
       'Driver_Total_Young_Adult_Ages_24_29', 'EEA_Full_Coverage_Indicator_N',
       'EEA_Full_Coverage_Indicator_Y',
       'EEA_Liability_Coverage_Only_Indicator_N',
       'EEA_Liability_Coverage_Only_Indicator_Y',
       'EEA_Multi_Auto_Policies_Indicator_N',
       'EEA_Multi_Auto_Policies_Indicator_Y',
       'EEA_Packaged_Policy_Indicator_N', 'EEA_Packaged_Policy_Indicator_Y',
       'EEA_Policy_Tenure', 'Frequency',
       'Policy_Billing_Code_Direct Billed to Insured',
       'Policy_Billing_Code_P

<ipython-input-9-0e40a977e25f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.strip()
<ipython-input-9-0e40a977e25f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.strip()
<ipython-input-9-0e40a977e25f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

...

<ipython-input-9-0e40a977e25f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.strip()
<ipython-input-9-0e40a977e25f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.strip()
<ipython-input-9-0e40a977e25f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

Index(['Annual_Premium', 'Claim_Count', 'Driver_Maximum_Age',
       'Driver_Minimum_Age', 'Driver_Total_Adult_Ages_50_64',
       'Driver_Total_College_Ages_20_23', 'Driver_Total_Female',
       'Driver_Total_Low_Middle_Adult_Ages_30_39', 'Driver_Total_Male',
       'Driver_Total_Married', 'Driver_Total_Middle_Adult_Ages_40_49',
       'Driver_Total_Senior_Ages_65_69', 'Driver_Total_Single',
       'Driver_Total_Teenager_Age_15_19',
       'Driver_Total_Upper_Senior_Ages_70_plus',
       'Driver_Total_Young_Adult_Ages_24_29', 'EEA_Full_Coverage_Indicator_N',
       'EEA_Full_Coverage_Indicator_Y',
       'EEA_Liability_Coverage_Only_Indicator_N',
       'EEA_Liability_Coverage_Only_Indicator_Y',
       'EEA_Multi_Auto_Policies_Indicator_N',
       'EEA_Multi_Auto_Policies_Indicator_Y',
       'EEA_Packaged_Policy_Indicator_N', 'EEA_Packaged_Policy_Indicator_Y',
       'EEA_Policy_Tenure', 'Frequency',
       'Policy_Billing_Code_Direct Billed to Insured',
       'Policy_Billing_Code_P

<ipython-input-9-0e40a977e25f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.strip()
<ipython-input-9-0e40a977e25f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.strip()
<ipython-input-9-0e40a977e25f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

Index(['Annual_Premium', 'Claim_Count', 'Driver_Maximum_Age',
       'Driver_Minimum_Age', 'Driver_Total_Adult_Ages_50_64',
       'Driver_Total_College_Ages_20_23', 'Driver_Total_Female',
       'Driver_Total_Low_Middle_Adult_Ages_30_39', 'Driver_Total_Male',
       'Driver_Total_Married', 'Driver_Total_Middle_Adult_Ages_40_49',
       'Driver_Total_Senior_Ages_65_69', 'Driver_Total_Single',
       'Driver_Total_Teenager_Age_15_19',
       'Driver_Total_Upper_Senior_Ages_70_plus',
       'Driver_Total_Young_Adult_Ages_24_29', 'EEA_Full_Coverage_Indicator_N',
       'EEA_Full_Coverage_Indicator_Y',
       'EEA_Liability_Coverage_Only_Indicator_N',
       'EEA_Liability_Coverage_Only_Indicator_Y',
       'EEA_Multi_Auto_Policies_Indicator_N',
       'EEA_Multi_Auto_Policies_Indicator_Y',
       'EEA_Packaged_Policy_Indicator_N', 'EEA_Packaged_Policy_Indicator_Y',
       'EEA_Policy_Tenure', 'Frequency',
       'Policy_Billing_Code_Direct Billed to Insured',
       'Policy_Billing_Code_P

<ipython-input-9-0e40a977e25f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.strip()
<ipython-input-9-0e40a977e25f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.strip()


In [71]:
for key, value in test_portfolios_dict.items():
    print(key, value.shape)

portfolio_1 (1000, 57)
portfolio_2 (999, 57)
portfolio_3 (1000, 57)
portfolio_4 (997, 57)
portfolio_5 (998, 57)
portfolio_6 (1000, 57)
portfolio_7 (1000, 57)
portfolio_8 (998, 57)
portfolio_9 (998, 57)
portfolio_10 (998, 57)
portfolio_11 (1000, 57)
portfolio_12 (999, 57)
portfolio_13 (999, 57)
portfolio_14 (997, 57)
portfolio_15 (1000, 57)
portfolio_16 (999, 57)
portfolio_17 (998, 57)
portfolio_18 (999, 57)
portfolio_19 (998, 57)
portfolio_20 (997, 57)
portfolio_21 (1000, 57)
portfolio_22 (999, 57)
portfolio_23 (1000, 57)
portfolio_24 (1000, 57)
portfolio_25 (998, 57)
portfolio_26 (1000, 57)
portfolio_27 (998, 57)
portfolio_28 (999, 57)
portfolio_29 (998, 57)
portfolio_30 (998, 57)
portfolio_41 (1000, 57)
portfolio_42 (1000, 57)
portfolio_43 (998, 57)
portfolio_44 (999, 57)
portfolio_45 (999, 57)
portfolio_46 (999, 57)
portfolio_47 (998, 57)
portfolio_48 (1000, 57)
portfolio_49 (999, 57)
portfolio_50 (999, 57)
portfolio_51 (999, 57)
portfolio_52 (998, 57)
portfolio_53 (996, 57)
portfol

In [74]:
df_test_final = features_to_testing_df(test_portfolios_dict)

In [83]:
def generate_test_results(regressor, X_test, output_dir):
    # for each policy
    result_data = collections.OrderedDict()
    portfolio_ids_df = X_test.iloc[:, 0:1]
    X_test_features = X_test.iloc[:, 1:]

    y_pred = regressor.predict(np.nan_to_num(X_test_features.to_numpy()))
    loss_ratio = y_pred/X_test['Annual_Premium']
    loss_ratio_log = np.log(loss_ratio)
    loss_ratio_log_df = loss_ratio_log.to_frame(name='ln_LR')
    result_df = portfolio_ids_df.join(loss_ratio_log_df, how='outer')

    result_df.to_csv('output.csv', index=False)

In [84]:
generate_test_results(xg_regressor, df_test_final, '')

# **Model Testings**

In [ ]:
!pip install pycaret

***Classification 1***

In [ ]:
finalTrainSet

,Severity,Claim_Count,Frequency,Vehicle_Age_In_Years,Vehicle_Make_Year,Vehicle_Symbol,Vehicle_Collision_Coverage_Deductible,Driver_Minimum_Age,Driver_Maximum_Age,Policy_Installment_Term,...,EEA_Packaged_Policy_Indicator_N,EEA_Packaged_Policy_Indicator_Y,EEA_Full_Coverage_Indicator_N,EEA_Full_Coverage_Indicator_Y,SYS_Renewed_N,SYS_Renewed_Y,SYS_New_Business_N,SYS_New_Business_Y,Annual_Premium,Loss_Amount
0,23.457640,0.0100,0.020981,7.0290,1996.9070,11.3290,287.5300,46.0110,46.9380,6.2400,...,528.0,472.0,415.0,585.0,114.0,886.0,899.0,101.0,332930.72,23457.64
1,82.668465,0.0210,0.069974,7.0710,1996.7220,11.1650,286.0350,46.1300,47.1510,6.2520,...,519.0,481.0,411.0,589.0,107.0,893.0,885.0,115.0,322085.33,84824.42
2,68.077360,0.0310,0.061630,7.1070,1996.8150,11.2260,298.8160,45.8930,46.7810,6.2400,...,547.0,453.0,381.0,619.0,122.0,878.0,908.0,92.0,330619.89,79171.16
3,213.203280,0.0410,0.116551,6.9530,1996.8510,11.2520,311.0740,45.9080,46.8700,6.2100,...,529.0,471.0,371.0,629.0,115.0,885.0,911.0,89.0,338087.68,215832.60
4,275.532030,0.0510,0.277647,6.9950,1996.9020,11.2150,293.1590,46.2120,47.0660,6.2820,...,500.0,500.0,385.0,615.0,97.0,903.0,906.0,94.0,341355.87,287399.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,263.966588,0.0736,0.172877,7.0468,1996.7082,11.1952,294.6660,45.9366,46.9658,6.2568,...,2712.0,2288.0,1949.0,3051.0,606.0,4394.0,4489.0,511.0,1673609.72,1372520.68
546,421.483320,0.0832,0.193231,7.0520,1996.5778,11.1230,289.8030,46.5554,47.6494,6.2136,...,2640.0,2360.0,1955.0,3045.0,581.0,4419.0,4499.0,501.0,1656458.93,2161991.03
547,391.016284,0.0958,0.197250,6.9746,1996.9762,11.2066,303.7018,45.9078,46.9698,6.2652,...,2576.0,2424.0,1870.0,3130.0,602.0,4398.0,4513.0,487.0,1704193.09,2033748.56
548,471.094020,0.1062,0.245702,6.9960,1996.9936,11.3342,297.4880,46.0674,47.0946,6.2424,...,2570.0,2430.0,1892.0,3108.0,541.0,4459.0,4498.0,502.0,1703403.80,2501385.87


In [1]:
X_train = finalTrainSet.loc[:, finalTrainSet.columns != 'Loss_Amount']
y_train = finalTrainSet['Loss_Amount']

NameError: name 'finalTrainSet' is not defined